# ChIP-seq

In [2]:
import os
import pandas as pd
import sys
import numpy as np
import sys
import re
sys.path.insert(0, '../..')
import itertools

from genepy.utils import helper as h
from genepy.epigenetics import chipseq as chip
from genepy.epigenetics import plot as genepyPlot

import dalmatian as dm
import pyBigWig

from bokeh.plotting import *
from IPython.display import IFrame
import igv
from biomart import BiomartServer
import io

from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture

from natsort import os_sorted

output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# MYB_degraded_v2

Chipseq of MYB degron in MV411 cells at 1h of degradation with 0.5$\mu$M VHL (with Drosophilia spike-in control)
<br>paired end
<br>__second replicate of chipseq_MYB_degraded_v1__

In [5]:
! gsutil ls gs://transfer-amlproject

gs://transfer-amlproject/Hello_Mo_Test
gs://transfer-amlproject/201120_MP8439_fastq/
gs://transfer-amlproject/210331_MP8777_fastq/
gs://transfer-amlproject/210514_MP8847_fastq/
gs://transfer-amlproject/210722_MP9119_fastq/
gs://transfer-amlproject/210823_TL9266_fastq/
gs://transfer-amlproject/211012_MP9377_fastq/
gs://transfer-amlproject/211025_MP9377_fastq/
gs://transfer-amlproject/211108_MP9454_fastq/
gs://transfer-amlproject/211217_MP9543_fastq/
gs://transfer-amlproject/220506_MP10011_fastq/
gs://transfer-amlproject/220506_MP10046_fastq/
gs://transfer-amlproject/220526_MP10028_fastq/
gs://transfer-amlproject/220526_MP10066_fastq/
gs://transfer-amlproject/220602_MP10090_fastq/
gs://transfer-amlproject/Cobinding_additional/
gs://transfer-amlproject/IRF2BP2_degraded_v6/
gs://transfer-amlproject/MEF2D_degraded_v4/
gs://transfer-amlproject/MONOMAC_synMEF2CD/
gs://transfer-amlproject/RNP_MEIS1/
gs://transfer-amlproject/RNPv5/
gs://transfer-amlproject/additional_degraded_v1/
gs://transfer-

In [3]:
project = "MYB_degraded_v2"
loc = "220526_MP10066_fastq" # transfer AML folder
datatype = "chipseq"
project_dir = datatype+"_"+project

In [12]:
fastq = ! gsutil ls gs://transfer-amlproject/$loc/*.fastq.gz

for f in fastq:
    print(os.path.basename(f))

20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R1_001.fastq.gz
20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R2_001.fastq.gz
20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R1_001.fastq.gz
20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R2_001.fastq.gz
20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R1_001.fastq.gz
20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R2_001.fastq.gz
20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181_R1_001.fastq.gz
20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181_R2_001.fastq.gz
20220526_5_MYBdegron1h_MYB-VHL_r2_MP10066_S182_R1_001.fastq.gz
20220526_5_MYBdegron1h_MYB-VHL_r2_MP10066_S182_R2_001.fastq.gz
20220526_6_MYBdegron1h_MYC-VHL_r2_MP10066_S183_R1_001.fastq.gz
20220526_6_MYBdegron1h_MYC-VHL_r2_MP10066_S183_R2_001.fastq.gz
20220526_7_MYBdegron1h_MED1-VHL_r2_MP10066_S184_R1_001.fastq.gz
20220526_7_MYBdegron1h_MED1-VHL_r2_MP10066_S184_R2_001.fastq.gz
20220526_8_MYBdegron1h_PolII-VHL_r2_MP10066_S185_R1_001.fastq.gz
20220526_8_MYBdegron1h_PolII-VHL_r2_M

In [13]:
mkdir ../data/$project_dir/ && mkdir ../data/$project_dir/qc/ # make dir in /home/monika/AMLproject/data

In [14]:
! gsutil -m cp gs://transfer-amlproject/$loc/multiqc_report.html ../data/$project_dir/qc/
! gsutil -m cp -r gs://transfer-amlproject/$loc/Reports/ ../data/$project_dir/qc/
! gsutil -m cp -r gs://transfer-amlproject/$loc/multiqc_data/ ../data/$project_dir/qc/

Copying gs://transfer-amlproject/220526_MP10066_fastq/multiqc_report.html...
/ [1/1 files][  1.3 MiB/  1.3 MiB] 100% Done                                    
Operation completed over 1 objects/1.3 MiB.                                      
Copying gs://transfer-amlproject/220526_MP10066_fastq/Reports/20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R1_001_fastqc.html...
Copying gs://transfer-amlproject/220526_MP10066_fastq/Reports/20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/220526_MP10066_fastq/Reports/20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R2_001_fastqc.html...
Copying gs://transfer-amlproject/220526_MP10066_fastq/Reports/20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R2_001_fastqc.zip...
Copying gs://transfer-amlproject/220526_MP10066_fastq/Reports/20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R1_001_fastqc.html...
Copying gs://transfer-amlproject/220526_MP10066_fastq/Reports/20220526_2_MYBdegron1h_MYC-DMSO_r2_MP100

In [15]:
! gsutil -m cp gs://transfer-amlproject/$loc/*  gs://transfer-amlproject/$project_dir/

Omitting prefix "gs://transfer-amlproject/220526_MP10066_fastq/Reports/". (Did you mean to do cp -r?)
Omitting prefix "gs://transfer-amlproject/220526_MP10066_fastq/multiqc_data/". (Did you mean to do cp -r?)
Copying gs://transfer-amlproject/220526_MP10066_fastq/20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/220526_MP10066_fastq/20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/220526_MP10066_fastq/20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/220526_MP10066_fastq/20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/220526_MP10066_fastq/20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R1_001.fastq.gz [Content-Type=application/oc

## download data

In [17]:
! mkdir ../../data/$project_dir && mkdir ../../data/$project_dir/fastqs \
&& gsutil -m cp gs://transfer-amlproject/$project_dir/* ../../data/$project_dir/fastqs

Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R2_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R2_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R2_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181_R2_001.fastq.gz...
Co

In [18]:
! gsutil -m cp gs://transfer-amlproject/$project_dir/* ../../data/$project_dir/fastqs

Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R2_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R2_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R2_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v2/20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181_R2_001.fastq.gz...
Co

In [19]:
! rm ../../data/$project_dir/fastqs/multiqc_report.html

In [20]:
fastq_name_list = [i.split("/")[4].split(".")[0] for i in fastq]
len(fastq_name_list)
fastq_name_list

['20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R1_001',
 '20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R2_001',
 '20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R1_001',
 '20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R2_001',
 '20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R1_001',
 '20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R2_001',
 '20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181_R1_001',
 '20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181_R2_001',
 '20220526_5_MYBdegron1h_MYB-VHL_r2_MP10066_S182_R1_001',
 '20220526_5_MYBdegron1h_MYB-VHL_r2_MP10066_S182_R2_001',
 '20220526_6_MYBdegron1h_MYC-VHL_r2_MP10066_S183_R1_001',
 '20220526_6_MYBdegron1h_MYC-VHL_r2_MP10066_S183_R2_001',
 '20220526_7_MYBdegron1h_MED1-VHL_r2_MP10066_S184_R1_001',
 '20220526_7_MYBdegron1h_MED1-VHL_r2_MP10066_S184_R2_001',
 '20220526_8_MYBdegron1h_PolII-VHL_r2_MP10066_S185_R1_001',
 '20220526_8_MYBdegron1h_PolII-VHL_r2_MP10066_S185_R2_001']

__rename local fastqs__

In [21]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret_web.json', '~/.storage.json')
url = "https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U/edit?ts=5fab1071#gid=738732237"
gsheet = sheets.get(url).sheets[10].to_frame()

In [23]:
# get new sample names from AML sample tracker - Chip_Tracker_JK
idx = gsheet.index[gsheet['processing project'] == project] # get first row with project number
sample_info = gsheet.loc[idx[0]:, ["previous name", "name"]]
sample_info

,previous name,name
444,20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178,mp998-MV411_MYB_DMSO-MYB-r2
445,20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179,mp999-MV411_MYB_DMSO-MYC-r2
446,20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180,mp1000-MV411_MYB_DMSO-MED1-r2
447,20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181,mp1001-MV411_MYB_DMSO-POLII_total-r2
448,20220526_5_MYBdegron1h_MYB-VHL_r2_MP10066_S182,mp1002-MV411_MYB_VHL-MYB-r2
449,20220526_6_MYBdegron1h_MYC-VHL_r2_MP10066_S183,mp1003-MV411_MYB_VHL-MYC-r2
450,20220526_7_MYBdegron1h_MED1-VHL_r2_MP10066_S184,mp1004-MV411_MYB_VHL-MED1-r2
451,20220526_8_MYBdegron1h_PolII-VHL_r2_MP10066_S185,mp1005-MV411_MYB_VHL-POLII_total-r2


In [25]:
fastqs = ! ls ../../data/$project_dir/fastqs
fastqs

['20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R1_001.fastq.gz',
 '20220526_1_MYBdegron1h_MYB-DMSO_r2_MP10066_S178_R2_001.fastq.gz',
 '20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R1_001.fastq.gz',
 '20220526_2_MYBdegron1h_MYC-DMSO_r2_MP10066_S179_R2_001.fastq.gz',
 '20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R1_001.fastq.gz',
 '20220526_3_MYBdegron1h_MED1-DMSO_r2_MP10066_S180_R2_001.fastq.gz',
 '20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181_R1_001.fastq.gz',
 '20220526_4_MYBdegron1h_PolII-DMSO_r2_MP10066_S181_R2_001.fastq.gz',
 '20220526_5_MYBdegron1h_MYB-VHL_r2_MP10066_S182_R1_001.fastq.gz',
 '20220526_5_MYBdegron1h_MYB-VHL_r2_MP10066_S182_R2_001.fastq.gz',
 '20220526_6_MYBdegron1h_MYC-VHL_r2_MP10066_S183_R1_001.fastq.gz',
 '20220526_6_MYBdegron1h_MYC-VHL_r2_MP10066_S183_R2_001.fastq.gz',
 '20220526_7_MYBdegron1h_MED1-VHL_r2_MP10066_S184_R1_001.fastq.gz',
 '20220526_7_MYBdegron1h_MED1-VHL_r2_MP10066_S184_R2_001.fastq.gz',
 '20220526_8_MYBdegron1h_PolII-VHL_r2_MP10066_

In [30]:
# make dict with names
rename = dict(zip(sample_info["previous name"], sample_info["name"]))

# rename fastqs locally
for val in fastqs:
    rep  = val
    for old, new in rename.items():
        rep = rep.replace(old, new)
    if rep != val:
        ! mv ../../data/$project_dir/fastqs/$val ../../data/$project_dir/fastqs/$rep
        ! gsutil cp gs://transfer-amlproject/$project_dir/$val gs://amlproject/Chip/$project/fastqs/$rep

In [37]:
! gsutil cp ../../data/$project_dir/fastqs/* gs://amlproject/Chip/$project/fastqs

Copying file://../../data/chipseq_MYB_degraded_v2/fastqs/mp1000-MV411_MYB_DMSO-MED1-r2_R1_001.fastq.gz [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://../../data/chipseq_MYB_degraded_v2/fastqs/mp1000-MV411_MYB_DMSO-MED1-r2_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying file://../../d

In [32]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret_web.json', '~/.storage.json')
url = "https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U/edit?ts=5fab1071#gid=738732237"
gsheet = sheets.get(url).sheets[10].to_frame()

In [38]:
fastq = ! gsutil ls gs://amlproject/Chip/$project/fastqs/

In [47]:
fastq = os_sorted(fastq)
fastq

['gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp998-MV411_MYB_DMSO-MYB-r2_R1_001.fastq.gz',
 'gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp998-MV411_MYB_DMSO-MYB-r2_R2_001.fastq.gz',
 'gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp999-MV411_MYB_DMSO-MYC-r2_R1_001.fastq.gz',
 'gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp999-MV411_MYB_DMSO-MYC-r2_R2_001.fastq.gz',
 'gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp1000-MV411_MYB_DMSO-MED1-r2_R1_001.fastq.gz',
 'gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp1000-MV411_MYB_DMSO-MED1-r2_R2_001.fastq.gz',
 'gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp1001-MV411_MYB_DMSO-POLII_total-r2_R1_001.fastq.gz',
 'gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp1001-MV411_MYB_DMSO-POLII_total-r2_R2_001.fastq.gz',
 'gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp1002-MV411_MYB_VHL-MYB-r2_R1_001.fastq.gz',
 'gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp1002-MV411_MYB_VHL-MYB-r2_R2_001.fastq.gz',
 'gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp1003-MV411

In [64]:
# design matrix
df = {
"fastq_1": [],
"fastq_2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in h.grouped(fastq, 2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('-')[0]]
    df['group'].append("MV4"+row['name'].values[0].split('-r')[0].split('-MV4')[1])
    # returned error when using replicate 2 without replicate 1 in the same run
    #df['replicate'].append(row['name'].values[0].split('-r')[1])
    df['replicate'].append(1)
    df['fastq_1'].append(val[0])
    df['fastq_2'].append(val[1])
    df['antibody'].append(row['protein'].values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq_1'].append('gs://amlproject/Chip/fastqs/paired_end/mp845-MV411-INPUT-r2_R1.fastq.gz')
df['fastq_2'].append('gs://amlproject/Chip/fastqs/paired_end/mp845-MV411-INPUT-r2_R2.fastq.gz')
df['antibody'].append("")
df['control'].append("")
df = pd.DataFrame(df)

In [65]:
df

,fastq_1,fastq_2,antibody,group,replicate,control
0,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,MYB,MV411_MYB_DMSO-MYB,1,INPUT
1,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,MYC,MV411_MYB_DMSO-MYC,1,INPUT
2,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,MED1,MV411_MYB_DMSO-MED1,1,INPUT
3,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,POLII_total,MV411_MYB_DMSO-POLII_total,1,INPUT
4,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,MYB,MV411_MYB_VHL-MYB,1,INPUT
5,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,MYC,MV411_MYB_VHL-MYC,1,INPUT
6,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,MED1,MV411_MYB_VHL-MED1,1,INPUT
7,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,POLII_total,MV411_MYB_VHL-POLII_total,1,INPUT
8,gs://amlproject/Chip/fastqs/paired_end/mp845-M...,gs://amlproject/Chip/fastqs/paired_end/mp845-M...,,INPUT,1,


had to set all replicates to 1 instead of 2 for nextflow to run

In [66]:
# nextflow CSV
design_file = "../nextflow/"+project+"_design.csv"
design_file_path = os.path.abspath(design_file)
print(design_file_path)
# saved with column order as: group, replicate, fastq_1, fastq_2, antibody, control
df[df.columns[[3,4,0,1,2,5]]].to_csv(design_file_path,index=False)

/home/monika/AMLproject/nextflow/MYB_degraded_v2_design.csv


In [67]:
df = pd.read_csv(design_file) # read in design file
df = df.replace(np.nan, '', regex=True)
df = df[["fastq_1", "fastq_2", "antibody", "group", "replicate", "control"]]
df.iloc[:1,:]

,fastq_1,fastq_2,antibody,group,replicate,control
0,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v2/fastqs/mp...,MYB,MV411_MYB_DMSO-MYB,1,INPUT


## run nextflow

__about__

0. you need to have a google project set up with a billing account
1. you need to activte your APIs this way: https://cloud.google.com/life-sciences/docs/tutorials/nextflow?hl=fr
2. nextflow needs to be installed with this installation command `export NXF_MODE=google && curl https://get.nextflow.io | bash`

In [55]:
! cd /home/monika && sudo ./nextflow log ## to get access to the previous runs

TIMESTAMP          	DURATION     	RUN NAME         	STATUS	REVISION ID	SESSION ID                          	COMMAND                                                                                                                                                                                                                                                                                                                                                                                                          
2021-07-08 20:08:32	-            	hopeful_albattani	-     	fb4924dd0d 	117e88ad-5b5f-4ad3-a368-4ebee2f804c8	nextflow run monikaperez/chipseq --paired_end --seq_center DFCI --email jkobject@gmail.com --input AMLproject/nextflow/MEF2D_degraded_v4_design.csv --genome GRCh38 --max_cpus 16 -profile mpcloud -w 'gs://workamlproject/MEF2Cv4res' --spiking --skip_diff_analysis --outdir 'gs://workamlproject/MEF2Cv4' --tracedir AMLproject/nextflow/MEF2Cdegradedv4/                                  

In [78]:
! cd /home/monika && sudo ./nextflow log ## to get access to the previous runs

TIMESTAMP          	DURATION     	RUN NAME         	STATUS	REVISION ID	SESSION ID                          	COMMAND                                                                                                                                                                                                                                                                                                                                                                                                          
2021-07-08 20:08:32	-            	hopeful_albattani	-     	fb4924dd0d 	117e88ad-5b5f-4ad3-a368-4ebee2f804c8	nextflow run monikaperez/chipseq --paired_end --seq_center DFCI --email jkobject@gmail.com --input AMLproject/nextflow/MEF2D_degraded_v4_design.csv --genome GRCh38 --max_cpus 16 -profile mpcloud -w 'gs://workamlproject/MEF2Cv4res' --spiking --skip_diff_analysis --outdir 'gs://workamlproject/MEF2Cv4' --tracedir AMLproject/nextflow/MEF2Cdegradedv4/                                  

In [ ]:
nextflow run monikaperez/chipseq 
--paired_end 
--seq_center DFCI 
--email monikawenperez@gmail.com 
--input /home/monika/AMLproject/nextflow/MYB_degraded_v2_design.csv 
--genome GRCh38 
--max_cpus 16 
-profile mpcloud 
-w 'gs://workamlproject/MYB_degraded_v2res' 
--outdir 'gs://workamlproject/MYB_degraded_v2' 
--spiking 
--tracedir AMLproject/nextflow/MYB_degraded_v2 
--skip_diff_analysis 


In [72]:
# write nextflow command to string
export_header = "export NXF_mode=google && sudo"
nextflow_call = "./nextflow run monikaperez/chipseq" # run outside of AMLproject dir
endedness = "--paired_end"
seq_center = "--seq_center " + "'DFCI'"
email = "--email " + "'monikawenperez@gmail.com'"
design_input = "--input " + design_file_path
genome = "--genome " + "GRCh38"
max_cpus = "--max_cpus " + str(16)
profile = "-profile " + "mpcloud"
work_dir = "-w " + "gs://workamlproject/{}res".format(project)
outdir = "--outdir "+"gs://workamlproject/"+project
spiking = "--spiking"
tracedir = "--tracedir AMLproject/nextflow/"+project
#peak_type = "--narrow_peak"

nextflow_args = [export_header, nextflow_call, endedness,
                 seq_center, email, design_input, 
                 genome, max_cpus, profile, 
                 work_dir, outdir, spiking, tracedir,
                 "--skip_diff_analysis"]

# process chips
print("Copy and paste string into command line:\nRun from outside of AMLproject. May want to run in screen.\n")
print("\n".join(nextflow_args))

Copy and paste string into command line:
Run from outside of AMLproject. May want to run in screen.

export NXF_mode=google && sudo
./nextflow run monikaperez/chipseq
--paired_end
--seq_center 'DFCI'
--email 'monikawenperez@gmail.com'
--input /home/monika/AMLproject/nextflow/MYB_degraded_v2_design.csv
--genome GRCh38
--max_cpus 16
-profile mpcloud
-w gs://workamlproject/MYB_degraded_v2res
--outdir gs://workamlproject/MYB_degraded_v2
--spiking
--tracedir AMLproject/nextflow/MYB_degraded_v2
--skip_diff_analysis


In [73]:
print(" ".join(nextflow_args))

export NXF_mode=google && sudo ./nextflow run monikaperez/chipseq --paired_end --seq_center 'DFCI' --email 'monikawenperez@gmail.com' --input /home/monika/AMLproject/nextflow/MYB_degraded_v2_design.csv --genome GRCh38 --max_cpus 16 -profile mpcloud -w gs://workamlproject/MYB_degraded_v2res --outdir gs://workamlproject/MYB_degraded_v2 --spiking --tracedir AMLproject/nextflow/MYB_degraded_v2 --skip_diff_analysis


move results to main bucket

In [79]:
! gsutil -m cp -r gs://workamlproject/$project/* gs://amlproject/Chip/$project/

Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam [Content-Type=application/octet-s

Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_DMSO-POLII_total_R1.plotFingerprint.raw.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MED1_R1.plotFingerprint.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MED1_R1.plotFingerprint.qcmetrics.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MED1_R1.plotFingerprint.raw.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MYB_R1.plotFingerprint.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MYB_R1.p

Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MED1_R1.plotProfile.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MYB_R1.computeMatrix.mat.gz [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MYB_R1.computeMatrix.vals.mat.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MYB_R1.plotHeatmap.mat.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MYB_R1.plotHeatmap.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MYB_R1.plotProfile.pdf [Content-Type=application/octet-stream

Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_VHL-POLII_total_R1_peaks.xls [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.annotatePeaks.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.bed [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.boolean.annotatePeaks.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.boolean.intersect.plot.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.boolean.intersect.txt [Conten

Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/qc/MV411_MYB_VHL-MYB_R1_peaks.count_mqc.tsv [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/qc/MV411_MYB_VHL-MYC_R1_peaks.FRiP_mqc.tsv [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/qc/MV411_MYB_VHL-MYC_R1_peaks.count_mqc.tsv [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/qc/MV411_MYB_VHL-POLII_total_R1_peaks.FRiP_mqc.tsv [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/qc/MV411_MYB_VHL-POLII_total_R1_peaks.count_mqc.tsv [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/qc/macs_annotatePeaks.plots.pdf [Content-Type=application/octet-stream]...
Copying gs://w

...

Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/picard_metrics/MV411_MYB_VHL-MYB_R1.mLb.clN.CollectMultipleMetrics.quality_distribution_metrics [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/picard_metrics/MV411_MYB_VHL-MYB_R1.mLb.mkD.MarkDuplicates.metrics.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/picard_metrics/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.alignment_summary_metrics [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/picard_metrics/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.base_distribution_by_cycle_metrics [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/picard_metrics/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.insert_size_metrics [Content-Type=application/octet-stream]...
Copying gs://workamlprojec

Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/picard_metrics/pdf/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.base_distribution_by_cycle.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/picard_metrics/pdf/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.insert_size_histogram.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/picard_metrics/pdf/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.quality_by_cycle.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/picard_metrics/pdf/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.quality_distribution.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/picard_metrics/pdf/MV411_MYB_VHL-POLII_total_R1.mLb.clN.CollectMultipleMetrics.base_distribution_by_cycle.pdf [Content-Type=applica

Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/samtools_stats/MV411_MYB_DMSO-POLII_total_R1.mLb.clN.sorted.bam.idxstats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/samtools_stats/MV411_MYB_DMSO-POLII_total_R1.mLb.clN.sorted.bam.stats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/samtools_stats/MV411_MYB_DMSO-POLII_total_R1.mLb.mkD.sorted.bam.flagstat [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/samtools_stats/MV411_MYB_DMSO-POLII_total_R1.mLb.mkD.sorted.bam.idxstats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/samtools_stats/MV411_MYB_DMSO-POLII_total_R1.mLb.mkD.sorted.bam.stats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/bwa/mergedLibrary/samtools_stats/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.ba

Copying gs://workamlproject/MYB_degraded_v2/droso_aligned/scale/MV411_MYB_VHL-MYB_R1_T1_scaling.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/droso_aligned/scale/MV411_MYB_VHL-MYC_R1_T1_scaling.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/droso_aligned/scale/MV411_MYB_VHL-POLII_total_R1_T1_scaling.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/fastqc/INPUT_R1_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/fastqc/INPUT_R1_T1_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/fastqc/MV411_MYB_DMSO-MED1_R1_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/fastqc/MV411_MYB_DMSO-MED1_R1_T1_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/fastqc/MV41

Copying gs://workamlproject/MYB_degraded_v2/trim_galore/fastqc/MV411_MYB_VHL-MED1_R1_T1_2_val_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/trim_galore/fastqc/MV411_MYB_VHL-MYB_R1_T1_1_val_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/trim_galore/fastqc/MV411_MYB_VHL-MYB_R1_T1_2_val_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/trim_galore/fastqc/MV411_MYB_VHL-MYC_R1_T1_1_val_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/trim_galore/fastqc/MV411_MYB_VHL-MYC_R1_T1_2_val_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/trim_galore/fastqc/MV411_MYB_VHL-POLII_total_R1_T1_1_val_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v2/trim_galore/fastqc/MV411_MYB_VHL-POLII_total_R1

## get counts for droso scales

In [80]:
# get unique mapped reads(droso)
! gsutil cat gs://amlproject/Chip/$project/droso_aligned/counts/*
! gsutil ls gs://amlproject/Chip/$project/droso_aligned/counts/

10383270
12131434
16519284
721324
11504934
13509484
19955952
8935846
13246418
gs://amlproject/Chip/MYB_degraded_v2/droso_aligned/counts/INPUT_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v2/droso_aligned/counts/MV411_MYB_DMSO-MED1_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v2/droso_aligned/counts/MV411_MYB_DMSO-MYB_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v2/droso_aligned/counts/MV411_MYB_DMSO-MYC_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v2/droso_aligned/counts/MV411_MYB_DMSO-POLII_total_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v2/droso_aligned/counts/MV411_MYB_VHL-MED1_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v2/droso_aligned/counts/MV411_MYB_VHL-MYB_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v2/droso_aligned/counts/MV411_MYB_VHL-MYC_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v2/droso_aligned/counts/MV411_MYB_VHL-POLII_total_R1_T1_counts.txt


In [82]:
counts = ! gsutil cat gs://amlproject/Chip/$project/droso_aligned/counts/*
samples = ! gsutil ls gs://amlproject/Chip/$project/droso_aligned/counts/

# get unique mapped reads(droso) to sample
for c,s in zip(counts, samples):
    print(s.split("/")[-1]+" : "+c)

INPUT_R1_T1_counts.txt : 10383270
MV411_MYB_DMSO-MED1_R1_T1_counts.txt : 12131434
MV411_MYB_DMSO-MYB_R1_T1_counts.txt : 16519284
MV411_MYB_DMSO-MYC_R1_T1_counts.txt : 721324
MV411_MYB_DMSO-POLII_total_R1_T1_counts.txt : 11504934
MV411_MYB_VHL-MED1_R1_T1_counts.txt : 13509484
MV411_MYB_VHL-MYB_R1_T1_counts.txt : 19955952
MV411_MYB_VHL-MYC_R1_T1_counts.txt : 8935846
MV411_MYB_VHL-POLII_total_R1_T1_counts.txt : 13246418


use the total counts in gs://amlproject/Chip/MYB_degraded_v1/multiqc/broadPeak/multiqc_report.html
<br>use MERGED LIB: SAMTools (filtered) > Percent Mapped data [samtools_alignment_plot-2]
https://storage.cloud.google.com/amlproject/Chip/MYB_degraded_v2/multiqc/broadPeak/multiqc_report.html
<br>manually enter total counts in "total" column in AML sample tracker > Chip_Tracker_JK

In [81]:
# copy bams, bigwigs, and macs data to local comp
h.createFoldersFor('../../data/chipseq_'+project+'/bwa/mergedLibrary/')
! gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/*.ba* ../../data/chipseq_$project/bwa/mergedLibrary/
! gsutil -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/bigwig/ ../../data/chipseq_$project/bwa/mergedLibrary/
! gsutil -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/macs/ ../../data/chipseq_$project/bwa/mergedLibrary/

Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-POLII_total_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/MYB_degr

Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.saf...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.bed...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.boolean.annotatePeaks.txt...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.boolean.intersect.plot.pdf...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.boolean.intersect.txt...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.boolean.txt...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/co

Computing scales from AML sample tracker sheet
* using drosophilia ratio (ratio to droso)
* DMSO: scale = treatment / DMSO
* treatment: scale = DMSO / treatment
* anything above 1 is set to 1 so we are not scaling twice
* DMSO > VHL: DMSO = VHL/DMSO & VHL = 1 (ratio to droso)
* DMSO < VHL: DMSO = 1 & VHL = DMSO/VHL (ratio to droso)

In [89]:
scales = [ #same as file list order
    1.00, #MV411_MYB_DMSO MED1 R1
    1.00, #MV411_MYB_DMSO MYB R1
    1.00, #MV411_MYB_DMSO MYC R1
    0.88, #MV411_MYB_DMSO POLII_total R1
    
    0.94, #MV411_MYB_VHL MED1 R1
    0.63, #MV411_MYB_VHL MYB R1
    0.11, #MV411_MYB_VHL MYC R1
    1.00  #MV411_MYB_VHL POLII_total R1
]

In [84]:
bams = ! ls ../../data/chipseq_$project/bwa/mergedLibrary/*.bam
bams_short = list(i.split("/")[6] for i in bams)
bams_short

['INPUT_R1.mLb.clN.sorted.bam',
 'MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam',
 'MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam',
 'MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam',
 'MV411_MYB_DMSO-POLII_total_R1.mLb.clN.sorted.bam',
 'MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam',
 'MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam',
 'MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam',
 'MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam']

In [85]:
names = ["MED1_R1", "MYB_R1", "MYC_R1", "POLII_total_R1"]

## on scaled data

In [86]:
! mkdir ../../data/chipseq_$project/diffPeaks/ && ! mkdir ../../data/chipseq_$project/diffData/

In [87]:
wigs = ! ls ../../data/chipseq_$project/bwa/mergedLibrary/bigwig/*.bigWig
wigs

['../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/INPUT_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MED1_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MYB_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MYC_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-POLII_total_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MED1_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MYB_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MYC_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-POLII_total_R1.bigWig']

### diffPeak on scaled data (full reprocessing)

__fullDiffPeak__
<br>uses macs3 to call differential peak binding from 2 bam files and their control
<br>(can also provide some spike-in scaling info)
* compute avg fragment size (macs3 predictd -i)
* macs3 callpeak -B -t bam1 -c control1 --nomodel --extsize predicted_fragment_length -n name1 --outdir directory -f endedness

__diffPeak__
<br>calls MACS2 bdgdiff

In [90]:
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    print(i)
    bam1 = bams[1+i]
    bam2 = bams[1+len(names)+i]
    print(bams_short[1+i])
    print(bams_short[1+len(names)+i])
    print([scales[i], scales[len(names)+i]])
    print("\n")

0
MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam
[1.0, 0.94]


1
MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam
[1.0, 0.63]


2
MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam
[1.0, 0.11]


3
MV411_MYB_DMSO-POLII_total_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam
[0.88, 1.0]




In [91]:
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    bam1 = bams[1+i]
    bam2 = bams[1+len(names)+i]
    print(bams_short[1+i])
    print(bams_short[1+len(names)+i])
    print(await chip.fullDiffPeak(bam1, bam2, control1 = bams[0], 
                            scaling = [scales[i], scales[len(names)+i]], 
                            directory = "../../data/chipseq_"+project+"/diffData/", 
                            res_directory = "../../data/chipseq_"+project+"/diffPeaks/", 
                            pairedend=True))
    print("\n")

MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam
doing diff from ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam and ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam
computing the fragment avg size
206
computing the scaling values
b'INFO  @ Mon, 13 Jun 2022 15:30:02: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 206 -n MV411_MYB_DMSO-MED1_R1 --outdir ../../data/chipseq_MYB_degraded_v2/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_DMSO-MED1_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.

b'INFO  @ Mon, 13 Jun 2022 15:49:56: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 206 -n MV411_MYB_VHL-MED1_R1 --outdir ../../data/chipseq_MYB_degraded_v2/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-MED1_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller dataset.\n# Range for c

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-MED1_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-MED1_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-MED1_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-MED1_R1_control_lambda.bdg --d1 24823039 --d2 27444726 -g 60 -l 206 --o-prefix MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v2/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Mon, 13 Jun 2022 16:09:54: Read and build treatment 1 bedGraph... \nINFO  @ Mon, 13 Jun 2022 16:10:29: Read and build control 1 bedGraph... \nINFO  @ Mon, 13 Jun 2022 16:12:56: Read and build treatment 2 bedGraph... \nINFO  @ Mon, 13 Jun 2022 16:13:30: Read and build control 2 bedGraph... \nINFO  @ Mon, 13 Jun 2022 16:21:52: Write peaks... \nINFO  @ Mon, 13 Jun 2022 16:21:52: 

b'INFO  @ Mon, 13 Jun 2022 16:45:54: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 218 -n MV411_MYB_VHL-MYB_R1 --outdir ../../data/chipseq_MYB_degraded_v2/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-MYB_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller dataset.\n# Range for calcu

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-MYB_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-MYB_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-MYB_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-MYB_R1_control_lambda.bdg --d1 30669533 --d2 31376461 -g 60 -l 218 --o-prefix MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MYB_VHL-MYB_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v2/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Mon, 13 Jun 2022 17:05:31: Read and build treatment 1 bedGraph... \nINFO  @ Mon, 13 Jun 2022 17:06:14: Read and build control 1 bedGraph... \nINFO  @ Mon, 13 Jun 2022 17:08:50: Read and build treatment 2 bedGraph... \nINFO  @ Mon, 13 Jun 2022 17:09:17: Read and build control 2 bedGraph... \nINFO  @ Mon, 13 Jun 2022 17:17:41: Write peaks... \nINFO  @ Mon, 13 Jun 2022 17:17:41: Done \

b'INFO  @ Mon, 13 Jun 2022 17:39:37: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 216 -n MV411_MYB_VHL-MYC_R1 --outdir ../../data/chipseq_MYB_degraded_v2/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-MYC_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller dataset.\n# Range for calcu

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-MYC_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-MYC_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-MYC_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-MYC_R1_control_lambda.bdg --d1 18791168 --d2 197135436 -g 60 -l 216 --o-prefix MV411_MYB_DMSO-MYC_R1_treat_pileup_vs_MV411_MYB_VHL-MYC_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v2/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Mon, 13 Jun 2022 18:00:30: Read and build treatment 1 bedGraph... \nINFO  @ Mon, 13 Jun 2022 18:00:56: Read and build control 1 bedGraph... \nINFO  @ Mon, 13 Jun 2022 18:03:33: Read and build treatment 2 bedGraph... \nINFO  @ Mon, 13 Jun 2022 18:04:02: Read and build control 2 bedGraph... \nINFO  @ Mon, 13 Jun 2022 18:12:13: Write peaks... \nINFO  @ Mon, 13 Jun 2022 18:12:13: Done 

b'INFO  @ Mon, 13 Jun 2022 18:36:46: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 208 -n MV411_MYB_VHL-POLII_total_R1 --outdir ../../data/chipseq_MYB_degraded_v2/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-POLII_total_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards sma

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-POLII_total_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-POLII_total_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-POLII_total_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-POLII_total_R1_control_lambda.bdg --d1 36212396 --d2 42425570 -g 60 -l 208 --o-prefix MV411_MYB_DMSO-POLII_total_R1_treat_pileup_vs_MV411_MYB_VHL-POLII_total_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v2/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Mon, 13 Jun 2022 19:00:35: Read and build treatment 1 bedGraph... \nINFO  @ Mon, 13 Jun 2022 19:01:19: Read and build control 1 bedGraph... \nINFO  @ Mon, 13 Jun 2022 19:03:52: Read and build treatment 2 bedGraph... \nINFO  @ Mon, 13 Jun 2022 19:04:47: Read and build control 2 bedGraph... \nINFO  @ Mon, 13 Jun 2022 19:14:10: Write peak

In [92]:
initscales = ! cat ../../data/chipseq_$project/bwa/mergedLibrary/bigwig/scale/*.txt
initscales

['0.00744797',
 '0.0199491',
 '0.0161936',
 '0.0263249',
 '0.0156004',
 '0.0192915',
 '0.0251341',
 '0.0228619',
 '0.0117035']

In [93]:
rescales = [val*float(initscales[1+i]) for i, val in enumerate(scales)]
rescales

[0.0199491,
 0.0161936,
 0.0263249,
 0.013728352000000001,
 0.01813401,
 0.015834483,
 0.0025148090000000002,
 0.0117035]

In [97]:
chip.bigWigFrom(bams[1:], 
                genome='GRCh38',scaling=rescales,
                numthreads=8)

In [98]:
! mkdir ../../data/chipseq_$project/recalib_bigwig/
! mv bigwig/* ../../data/chipseq_$project/recalib_bigwig/
! gsutil -m cp -r ../../data/chipseq_$project/recalib_bigwig gs://amlproject/Chip/$project/

Copying file://../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-POLII_total_R1.bw [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-MYC_R1.bw [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MED1_R1.bw [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-MED1_R1.bw [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-POLII_total_R1.bw [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MYB_R1.bw [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MYC_R1.bw [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/reca

In [99]:
os.popen('for i in $(ls ../../data/chipseq_'+project+'/diffPeaks/*.bed); \
            do echo $(wc -l $i); \
            done').read().split('\n')

['1895 ../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup_c3.0_common.bed',
 '1768 ../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup_c3.0_cond1.bed',
 '9 ../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup_c3.0_cond2.bed',
 '14056 ../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MYB_VHL-MYB_R1_treat_pileup_c3.0_common.bed',
 '20480 ../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MYB_VHL-MYB_R1_treat_pileup_c3.0_cond1.bed',
 '76 ../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MYB_VHL-MYB_R1_treat_pileup_c3.0_cond2.bed',
 '1 ../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MYC_R1_treat_pileup_vs_MV411_MYB_VHL-MYC_R1_treat_pileup_c3.0_common.bed',
 '36314 ..

In [100]:
bed_counts = ! wc -l ../../data/chipseq_$project/diffPeaks/*.bed

In [101]:
def getDiffPeakCounts(bed_counts, scaled, names):
    peak_types = ["common", "cond1", "cond2"]
    rows = []
    
    for i,s in enumerate(bed_counts[:-1]):
        counts = re.search(r'\d+', s).group()
        file = s.split("    ")[-1].split("/")[-1]
        sample_group = next(name for name in names if name in file)
        cond_dict = {"common": "COMMON", "cond1": "DMSO", "cond2": "VHL"}
        peak_cond = cond_dict.get(next(peak_type for peak_type in peak_types if peak_type in file))
        rows.append([scaled, sample_group, peak_cond, file, counts])
    
    return(pd.DataFrame(rows, columns=["SCALE", "SAMPLE", "PEAK_TYPE", "FILE", "N_PEAK"]))

In [102]:
df_diff_peak = getDiffPeakCounts(bed_counts, scaled="SCALED", names=names)

In [103]:
df_diff_peak.head()

,SCALE,SAMPLE,PEAK_TYPE,FILE,N_PEAK
0,SCALED,MED1_R1,COMMON,MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_M...,1895
1,SCALED,MED1_R1,DMSO,MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_M...,1768
2,SCALED,MED1_R1,VHL,MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_M...,9
3,SCALED,MYB_R1,COMMON,MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MY...,14056
4,SCALED,MYB_R1,DMSO,MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MY...,20480


In [104]:
bw = ! ls ../../data/chipseq_$project/recalib_bigwig/*
bw

['../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MED1_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MYB_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MYC_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-POLII_total_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-MED1_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-MYB_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-MYC_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-POLII_total_R1.bw']

In [105]:
! mkdir ../results/chipseq_$project/
! mkdir ../results/chipseq_$project/plots
! mkdir ../results/chipseq_$project/plots/heatmaps/

In [106]:
peaks = ! ls ../../data/chipseq_$project/bwa/mergedLibrary/macs/broadPeak/*.broadPeak
peaks

['../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_DMSO-MED1_R1_peaks.broadPeak',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_DMSO-MYB_R1_peaks.broadPeak',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_DMSO-MYC_R1_peaks.broadPeak',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_DMSO-POLII_total_R1_peaks.broadPeak',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_VHL-MED1_R1_peaks.broadPeak',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_VHL-MYB_R1_peaks.broadPeak',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_VHL-MYC_R1_peaks.broadPeak',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_VHL-POLII_total_R1_peaks.broadPeak']

### merging peaks VHL/DMSO

In [107]:
mpeaks = []
for i,val in enumerate(names):
    if i<0:
        continue
    print(val)
    dmso = peaks[i]
    vhl = peaks[i+len(names)]
    chip.simpleMergePeaks(pd.concat([chip.loadPeaks(dmso), chip.loadPeaks(vhl)])).to_csv(
        '../../data/chipseq_' + project + '/' + val + '_genomewide_merged.bed', sep='\t', header=False, index=False)
    mpeaks.append('../../data/chipseq_'+project+'/'+val+'_genomewide_merged.bed')

MED1_R1


/home/monika/genepy/genepy/epigenetics/chipseq.py:127: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  bindings = bindings.drop(5, 1).drop(4, 1).rename(columns={6:4, 7:5, 8:6, 9:7,})


0.09906474820143885

0.21446043165467626

0.4265467625899281

...

0.7044442068498521

0.7574396806728679

0.956532069330102

0.991137721705454

MYC_R1
POLII_total_R152743546


### GENOME WIDE comparison

In [108]:
dmsoname = "_DMSO"
condname = "_MYB_ko"

In [109]:
bw

['../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MED1_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MYB_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MYC_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-POLII_total_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-MED1_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-MYB_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-MYC_R1.bw',
 '../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-POLII_total_R1.bw']

In [116]:
for i, val in enumerate(bw):
    if i <0:
        continue
    print(os.path.basename(val))
    name = names[i-len(names)]+condname if i//len(names) else names[i]+dmsoname
    print(name)
    print(os.path.basename(mpeaks[i%len(names)]))
    genepyPlot.getPeaksAt(mpeaks[i%len(names)], bigwigs=val, bigwignames=name, 
                          peaknames=['Macs2_Peaks'], window=3000, folder="", title=name, numthreads=8, 
                          refpoint="center", name="../../data/chipseq_"+project+"/"+name+"_mat.pdf",
                          withDeeptools=True, torecompute=True, legendLoc="lower-left")

MV411_MYB_DMSO-MED1_R1.bw
MED1_R1_DMSO
MED1_R1_genomewide_merged.bed
CompletedProcess(args="computeMatrix reference-point -S ../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MED1_R1.bw  --referencePoint center --regionsFileName ../../data/chipseq_MYB_degraded_v2/MED1_R1_genomewide_merged.bed  --missingDataAsZero --outFileName ../../data/chipseq_MYB_degraded_v2/MED1_R1_DMSO_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 8 && plotHeatmap --matrixFile ../../data/chipseq_MYB_degraded_v2/MED1_R1_DMSO_mat.gz --outFileName ../../data/chipseq_MYB_degraded_v2/MED1_R1_DMSO_mat.pdf --refPointLabel center --legendLocation lower-left --regionsLabel Macs2_Peaks --plotTitle 'MED1_R1_DMSO'", returncode=0, stdout=b'', stderr=b'')
MV411_MYB_DMSO-MYB_R1.bw
MYB_R1_DMSO
MYB_R1_genomewide_merged.bed
CompletedProcess(args="computeMatrix reference-point -S ../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MYB_R1.bw  --referencePoint center --regionsFileName ../../

### making overlapping profiles

In [120]:
for i, val in enumerate(names):
    if i <0:
        continue
    val1 = "../../data/chipseq_"+project+"/"+val+dmsoname+"_mat.gz"
    val2 = "../../data/chipseq_"+project+"/"+val+condname+"_mat.gz"
    print(val)
    genepyPlot.makeProfiles(matx=[val1,val2], matnames=['DMSO','VHL'], title=val, 
                            refpoint="center", name="../../data/chipseq_"+project+"/"+val+"_combined_mat.pdf", 
                            legendLoc="lower-left")

MED1_R1
CompletedProcess(args='computeMatrixOperations relabel -m ../../data/chipseq_MYB_degraded_v2/MED1_R1_DMSO_mat.gz -o ../../data/chipseq_MYB_degraded_v2/MED1_R1_DMSO_mat.gz --groupLabels DMSO && computeMatrixOperations relabel -m ../../data/chipseq_MYB_degraded_v2/MED1_R1_MYB_ko_mat.gz -o ../../data/chipseq_MYB_degraded_v2/MED1_R1_MYB_ko_mat.gz --groupLabels VHL && computeMatrixOperations rbind -m ../../data/chipseq_MYB_degraded_v2/MED1_R1_DMSO_mat.gz ../../data/chipseq_MYB_degraded_v2/MED1_R1_MYB_ko_mat.gz -o ../../data/chipseq_MYB_degraded_v2/MED1_R1_combined_mat.gz && plotProfile --matrixFile ../../data/chipseq_MYB_degraded_v2/MED1_R1_combined_mat.gz --outFileName ../../data/chipseq_MYB_degraded_v2/MED1_R1_combined_mat.pdf --refPointLabel center --legendLocation lower-left --plotTitle MED1_R1', returncode=0, stdout=b'', stderr=b'')
MYB_R1
CompletedProcess(args='computeMatrixOperations relabel -m ../../data/chipseq_MYB_degraded_v2/MYB_R1_DMSO_mat.gz -o ../../data/chipseq_MYB_de

In [121]:
h.createFoldersFor("../results/chipseq_"+project+"/plots/scaled/heatmaps/")
! cp ../../data/chipseq_$project/*.pdf ../results/chipseq_$project/plots/scaled/heatmaps/

In [145]:
! cp ../../data/chipseq_$project/*.bed ../results/chipseq_$project/plots/scaled/heatmaps/

In [122]:
cond1peak = ! ls ../../data/chipseq_$project/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/chipseq_$project/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/chipseq_$project/diffPeaks/*common.bed

In [123]:
for i in range(int(len(bw)/2)):
    if i <0:
        continue
    # DMSO & VHL bw
    name1 = bw[i]
    name2 = bw[i+len(names)]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
   # for val in peak:
      #  chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name)
    genepyPlot.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], 
                          peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], 
                          window=3000, folder="", title=name, numthreads=8, refpoint="center", 
                          name='../../data/chipseq_'+project+'/diffPeaks/'+name+'_mat.pdf', 
                          withDeeptools=True, torecompute=False)

MED1_R1
CompletedProcess(args="computeMatrix reference-point -S ../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_DMSO-MED1_R1.bw ../../data/chipseq_MYB_degraded_v2/recalib_bigwig/MV411_MYB_VHL-MED1_R1.bw  --referencePoint center --regionsFileName ../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup_c3.0_cond1.bed ../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup_c3.0_common.bed  --missingDataAsZero --outFileName ../../data/chipseq_MYB_degraded_v2/diffPeaks/MED1_R1_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 8 && plotHeatmap --matrixFile ../../data/chipseq_MYB_degraded_v2/diffPeaks/MED1_R1_mat.gz --outFileName ../../data/chipseq_MYB_degraded_v2/diffPeaks/MED1_R1_mat.pdf --refPointLabel center --regionsLabel DMSO_peaks common --plotTitle 'MED1_R1'", returncode=0, stdout=b'', stderr=b'')
MYB_R1
CompletedProcess(args="computeMat

## on unscaled data

### diffPeak on unscaled data (full reprocessing)

In [124]:
bams

['../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-POLII_total_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam']

In [130]:
# on unscalled data
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    name1 = bams[1+i]
    name2 = bams[1+len(names)+i]
    print(os.path.basename(name1))
    print(os.path.basename(name2))
    # call diff peak binding w/ macs3
    print(await chip.fullDiffPeak(name1, name2, control1=bams[0], 
                                  directory = "../../data/chipseq_"+project+"/diffData_unscaled/", 
                                  res_directory = "../../data/chipseq_"+project+"/diffPeaks_unscaled/",
                                  pairedend=True))
    print("\n")

MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam
doing diff from ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam and ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam
computing the fragment avg size
206
computing the scaling values
b'INFO  @ Tue, 14 Jun 2022 15:14:50: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 206 -n MV411_MYB_DMSO-MED1_R1 --outdir ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_DMSO-MED1_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1

b'INFO  @ Tue, 14 Jun 2022 15:34:55: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 206 -n MV411_MYB_VHL-MED1_R1 --outdir ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-MED1_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller dataset.\n# Ra

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_DMSO-MED1_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_DMSO-MED1_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MED1_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MED1_R1_control_lambda.bdg --d1 24823039 --d2 25798043 -g 60 -l 206 --o-prefix MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v2/diffPeaks_unscaled/', returncode=0, stdout=b'', stderr=b'INFO  @ Tue, 14 Jun 2022 15:56:25: Read and build treatment 1 bedGraph... \nINFO  @ Tue, 14 Jun 2022 15:56:59: Read and build control 1 bedGraph... \nINFO  @ Tue, 14 Jun 2022 15:59:28: Read and build treatment 2 bedGraph... \nINFO  @ Tue, 14 Jun 2022 16:00:03: Read and build control 2 bedGraph... \nINFO  @ Tue, 14 Jun 2022 16:08:33: Write p

b'INFO  @ Tue, 14 Jun 2022 16:32:57: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 218 -n MV411_MYB_VHL-MYB_R1 --outdir ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-MYB_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller dataset.\n# Range 

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_DMSO-MYB_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_DMSO-MYB_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MYB_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MYB_R1_control_lambda.bdg --d1 30669533 --d2 19767171 -g 60 -l 218 --o-prefix MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MYB_VHL-MYB_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v2/diffPeaks_unscaled/', returncode=0, stdout=b'', stderr=b'INFO  @ Tue, 14 Jun 2022 16:53:33: Read and build treatment 1 bedGraph... \nINFO  @ Tue, 14 Jun 2022 16:54:15: Read and build control 1 bedGraph... \nINFO  @ Tue, 14 Jun 2022 16:56:47: Read and build treatment 2 bedGraph... \nINFO  @ Tue, 14 Jun 2022 16:57:14: Read and build control 2 bedGraph... \nINFO  @ Tue, 14 Jun 2022 17:05:35: Write peaks..

b'INFO  @ Tue, 14 Jun 2022 17:27:41: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 216 -n MV411_MYB_VHL-MYC_R1 --outdir ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-MYC_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller dataset.\n# Range 

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_DMSO-MYC_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_DMSO-MYC_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MYC_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MYC_R1_control_lambda.bdg --d1 18791168 --d2 21684898 -g 60 -l 216 --o-prefix MV411_MYB_DMSO-MYC_R1_treat_pileup_vs_MV411_MYB_VHL-MYC_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v2/diffPeaks_unscaled/', returncode=0, stdout=b'', stderr=b'INFO  @ Tue, 14 Jun 2022 17:47:44: Read and build treatment 1 bedGraph... \nINFO  @ Tue, 14 Jun 2022 17:48:10: Read and build control 1 bedGraph... \nINFO  @ Tue, 14 Jun 2022 17:50:38: Read and build treatment 2 bedGraph... \nINFO  @ Tue, 14 Jun 2022 17:51:07: Read and build control 2 bedGraph... \nINFO  @ Tue, 14 Jun 2022 17:59:08: Write peaks..

b'INFO  @ Tue, 14 Jun 2022 18:24:07: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 208 -n MV411_MYB_VHL-POLII_total_R1 --outdir ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-POLII_total_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled to

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_DMSO-POLII_total_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_DMSO-POLII_total_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-POLII_total_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-POLII_total_R1_control_lambda.bdg --d1 31866909 --d2 42425570 -g 60 -l 208 --o-prefix MV411_MYB_DMSO-POLII_total_R1_treat_pileup_vs_MV411_MYB_VHL-POLII_total_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v2/diffPeaks_unscaled/', returncode=0, stdout=b'', stderr=b'INFO  @ Tue, 14 Jun 2022 18:48:15: Read and build treatment 1 bedGraph... \nINFO  @ Tue, 14 Jun 2022 18:49:01: Read and build control 1 bedGraph... \nINFO  @ Tue, 14 Jun 2022 18:51:41: Read and build treatment 2 bedGraph... \nINFO  @ Tue, 14 Jun 2022 18:52:38: Read and build control 2 bedGraph... \n

In [131]:
bed_counts_unscaled = ! wc -l ../../data/chipseq_$project/diffPeaks_unscaled/*.bed

In [133]:
df_diff_peak = df_diff_peak.append(getDiffPeakCounts(bed_counts_unscaled, scaled="UNSCALED", 
                                                     names=names), ignore_index=True)

In [134]:
print(df_diff_peak.shape)
df_diff_peak

(24, 5)


,SCALE,SAMPLE,PEAK_TYPE,FILE,N_PEAK
0,SCALED,MED1_R1,COMMON,MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_M...,1895
1,SCALED,MED1_R1,DMSO,MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_M...,1768
2,SCALED,MED1_R1,VHL,MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_M...,9
3,SCALED,MYB_R1,COMMON,MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MY...,14056
4,SCALED,MYB_R1,DMSO,MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MY...,20480
5,SCALED,MYB_R1,VHL,MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MY...,76
6,SCALED,MYC_R1,COMMON,MV411_MYB_DMSO-MYC_R1_treat_pileup_vs_MV411_MY...,1
7,SCALED,MYC_R1,DMSO,MV411_MYB_DMSO-MYC_R1_treat_pileup_vs_MV411_MY...,36314
8,SCALED,MYC_R1,VHL,MV411_MYB_DMSO-MYC_R1_treat_pileup_vs_MV411_MY...,1
9,SCALED,POLII_total_R1,COMMON,MV411_MYB_DMSO-POLII_total_R1_treat_pileup_vs_...,40246


In [135]:
# diff peak count CSV
diff_peak_count_file = "../../data/chipseq_"+project+"/"+project+"_diff_peak_counts.csv"
diff_peak_count_file_path = os.path.abspath(diff_peak_count_file)
print(diff_peak_count_file_path)
df_diff_peak.to_csv(diff_peak_count_file_path, index=False)

/home/monika/data/chipseq_MYB_degraded_v2/MYB_degraded_v2_diff_peak_counts.csv


In [142]:
! cp $diff_peak_count_file ../results/chipseq_$project/

In [136]:
bw = ! ls ../../data/chipseq_$project/bwa/mergedLibrary/bigwig/*.bigWig
bw

['../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/INPUT_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MED1_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MYB_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MYC_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-POLII_total_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MED1_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MYB_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MYC_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-POLII_total_R1.bigWig']

### make overlapping profiles

In [137]:
cond1peak = ! ls ../../data/chipseq_$project/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/chipseq_$project/diffPeaks_unscaled/*cond2.bed
commonpeak = ! ls ../../data/chipseq_$project/diffPeaks_unscaled/*common.bed

In [141]:
for i in range(int(len(bw)/2)):
    if i <0:
        continue
    # DMSO & VHL bw
    name1 = bw[1+i]
    name2 = bw[1+i+len(names)]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
   # for val in peak:
      #  chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name)
    #print(os.path.basename(name1))
    #print(os.path.basename(name2))
    genepyPlot.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], 
                          peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], 
                          window=3000, folder="", title=name, numthreads=8, refpoint="center", 
                          name='../../data/chipseq_'+project+'/diffPeaks_unscaled/'+name+'_mat.pdf', 
                          withDeeptools=True, torecompute=True)

MED1_R1
CompletedProcess(args="computeMatrix reference-point -S ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MED1_R1.bigWig ../../data/chipseq_MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MED1_R1.bigWig  --referencePoint center --regionsFileName ../../data/chipseq_MYB_degraded_v2/diffPeaks_unscaled/MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup_c3.0_cond1.bed ../../data/chipseq_MYB_degraded_v2/diffPeaks_unscaled/MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup_c3.0_common.bed  --missingDataAsZero --outFileName ../../data/chipseq_MYB_degraded_v2/diffPeaks_unscaled/MED1_R1_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 8 && plotHeatmap --matrixFile ../../data/chipseq_MYB_degraded_v2/diffPeaks_unscaled/MED1_R1_mat.gz --outFileName ../../data/chipseq_MYB_degraded_v2/diffPeaks_unscaled/MED1_R1_mat.pdf --refPointLabel center --regionsLabel DMSO_peaks common --plotTitle 'MED1_R1'", retur

In [143]:
h.createFoldersFor('../results/chipseq_'+project+'/plots/unscaled/diffPeaks/')
! cp ../../data/chipseq_$project/diffPeaks_unscaled/*.pdf ../results/chipseq_$project/plots/unscaled/diffPeaks/

! gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/deepTools/**.pdf ../results/chipseq_$project/plots/

Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_DMSO-MED1_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_DMSO-MYB_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_DMSO-MYC_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_DMSO-POLII_total_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MED1_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MYB_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MYC_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/MYB_degraded_v2/bwa/merged

In [144]:
! gsutil -m cp -r ../../data/chipseq_$project/diffPeaks gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/chipseq_$project/diffData gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/chipseq_$project/diffPeaks_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/chipseq_$project/diffData_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/chipseq_$project/peakplot gs://amlproject/Chip/$project/

Copying file://../../data/chipseq_MYB_degraded_v2/diffPeaks/POLII_total_R1_mat.gz [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-POLII_total_R1_treat_pileup_vs_MV411_MYB_VHL-POLII_total_R1_treat_pileup_c3.0_common.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MYC_R1_treat_pileup_vs_MV411_MYB_VHL-MYC_R1_treat_pileup_c3.0_cond2.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffPeaks/MYB_R1_mat.gz [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup_c3.0_common.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffPeaks/MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MYB_VHL-MYB_R1_treat_pileup_c3.0_cond1.bed [Content-Type=applica

Copying file://../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-MYB_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-MYC_R1_treat_pileup.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-MYB_R1_control_lambda.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-MYB_R1_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_DMSO-MYC_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-MYC_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData/MV411_MYB_VHL-POLII_total_R1_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../../data/chipseq_MY

Copying file://../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_DMSO-MYC_R1_peaks.narrowPeak [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MED1_R1_control_lambda.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MYB_R1_control_lambda.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MYC_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MED1_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_DMSO-MYB_R1_treat_pileup.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MYB_degraded_v2/diffData_unscaled/MV411_MYB_VHL-MYB_R1_summits.bed [Conten

# Getting IGV paths

In [4]:
! gsutil ls gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig

gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig/INPUT_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MED1_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MYB_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MYC_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-POLII_total_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MED1_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MYB_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MYC_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-POLII_total_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/bigwig/scale/


In [5]:
! gsutil ls gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig

gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig/INPUT_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MED1_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MYB_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MYC_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-POLII_total_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MED1_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MYB_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MYC_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-POLII_total_R1.bigWig
gs://amlproject/Chip/MYB_degraded_v2/bwa/mergedLibrary/bigwig/scale/
